In [1]:
# ===============================================================
#  KAGGLE SETUP & IMPORTS
# ===============================================================
!pip install -q google-generativeai
!pip install -q sentence-transformers

import pandas as pd
import os
import time
import json
import torch
from kaggle_secrets import UserSecretsClient # To get our API key securely

import google.generativeai as genai
# Import the specific types for safety settings
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Imports for semantic search
from sentence_transformers import SentenceTransformer, util
import numpy as np

# --- Load API Key from Kaggle Secrets ---
try:
    user_secrets = UserSecretsClient()
    # Get the Google API key from the new secret
    secret_value = user_secrets.get_secret("GOOGLE_API_KEY") 
    genai.configure(api_key=secret_value)
    print("Google API key loaded successfully.")
    
    # Define robust safety settings
    safety_settings_config = [
        {
            "category": HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            "threshold": HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": HarmCategory.HARM_CATEGORY_HARASSMENT,
            "threshold": HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            "threshold": HarmBlockThreshold.BLOCK_NONE,
        },
        {
            "category": HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            "threshold": HarmBlockThreshold.BLOCK_NONE,
        },
    ]
    
    # Instantiate the Gemini model
    gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-05-20')
except Exception as e:
    print("Could not load Google API key or configure model. Make sure it's stored as a secret named 'GOOGLE_API_KEY'.")
    print(f"Error: {e}")
    # You might want to stop execution if the key isn't found
    # For a hackathon, we'll let it continue and fail on the API call
    gemini_model = None
    safety_settings_config = []


# --- Load Data ---
# Adjust 'competition-folder-name' to the actual folder name in /kaggle/input/
DATA_PATH = "/kaggle/input/kenya-clinical-reasoning-challenge20250407/"
train_df = pd.read_csv(DATA_PATH + 'train.csv')
test_df = pd.read_csv(DATA_PATH + 'test.csv')
sample_submission_df = pd.read_csv(DATA_PATH + 'SampleSubmission.csv')

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.0 MB/s eta 0:00:00


2025-06-10 07:07:19.684573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749539239.869946      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749539239.920785      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Google API key loaded successfully.
Train data shape: (400, 12)
Test data shape: (100, 7)


In [2]:
# ===============================================================
#  SEMANTIC SEARCH ENGINE INITIALIZATION
# ===============================================================
print("Initializing semantic search engine...")
# Using a lightweight but powerful model perfect for this task
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert all training prompts into semantic vectors (embeddings)
# This is done only once.
train_embeddings = embedding_model.encode(train_df['Prompt'].tolist(), convert_to_tensor=True)
print("Semantic search engine ready.")

def get_semantic_best_example(query_prompt: str) -> pd.Series:
    """Finds the single most semantically similar example from the training set."""
    query_embedding = embedding_model.encode(query_prompt, convert_to_tensor=True)
    
    # Calculate cosine similarity between the query and all training examples
    cosine_scores = util.cos_sim(query_embedding, train_embeddings)

    # Use torch.argmax and .item() to get a pure Python integer
    best_match_index = torch.argmax(cosine_scores).item()

    
    return train_df.iloc[best_match_index]


Initializing semantic search engine...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Semantic search engine ready.


In [3]:
# ===============================================================
#  "RELIABLE SCRIBE" API CALL FUNCTION
# ===============================================================

def create_structured_prompt(prompt_text: str) -> str:
    """Creates a one-shot prompt that instructs the model to output JSON."""
    example_row = get_semantic_best_example(prompt_text)
    
    # Create a structured JSON representation of the example's clinician response for the prompt
    # This part is tricky, as we need to parse the free text into a JSON structure.
    # We'll make a simplified assumption for the hackathon: we create a JSON-like string.
    # A more advanced version would use another LLM call to structure the example, but that's too slow.
    example_json_output = {
        "summary": "Example summary based on clinician text.",
        "diagnosis": "Example diagnosis based on clinician text.",
        "management_plan": example_row['Clinician'] # Put the full text here as a fallback
    }
    
    prompt = """You are an expert Kenyan clinician. Your task is to provide guidance based on a clinical vignette.
Your response MUST be a single, valid JSON object containing three keys: "summary", "diagnosis", and "management_plan".
Do not include any text or formatting outside of the JSON object.

--- EXAMPLE ---
Vignette: "{example_prompt}"
Clinician Response (JSON):
{example_json}

--- TASK ---
Vignette: "{task_prompt}"
Clinician Response (JSON):
""".format(
        example_prompt=example_row['Prompt'],
        example_json=json.dumps(example_json_output, indent=2), # Show a nicely formatted JSON example
        task_prompt=prompt_text
    )
    return prompt

def generate_structured_response(prompt_text: str) -> str:
    """Generates a structured JSON response and formats it into the final string."""
    full_prompt = create_structured_prompt(prompt_text)
    
    try:
        generation_config = genai.GenerationConfig(temperature=0.4, max_output_tokens=65536)
        response = gemini_model.generate_content(full_prompt, generation_config=generation_config, safety_settings=safety_settings_config)

        if response.parts:
            raw_text = response.text.strip()
            # Find the JSON part of the response, in case the model adds extra text like ```json
            json_start = raw_text.find('{')
            json_end = raw_text.rfind('}') + 1
            json_text = raw_text[json_start:json_end]

            # Attempt to parse the JSON
            data = json.loads(json_text)
            
            # Format the structured data into the final clinician string
            return f"summary: {data.get('summary', 'N/A')}\ndiagnosis: {data.get('diagnosis', 'N/A')}\nplan: {data.get('management_plan', 'N/A')}"
        else:
            return "Error: Model returned no response."

    except json.JSONDecodeError:
        # Fallback if the model output is not valid JSON
        return f"Error: Failed to parse JSON. Raw output: {raw_text}"
    except Exception as e:
        return f"Error: An unexpected API error occurred: {e}"

In [4]:
# ===============================================================
#  MAIN EXECUTION SCRIPT
# ===============================================================
if gemini_model:
    results = []
    print("Starting prediction process with 'Moonshot' architecture...")

    RPM = 10
    requests_this_minute = 0
    start_time = time.time()
    
    for row in test_df.itertuples():
        print(f"Processing row {row.Index + 1}/{len(test_df)}: Master_Index = {row.Master_Index}")

        if requests_this_minute >= RPM:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                sleep_time = 60.1 - elapsed_time # Add a small buffer
                print(f"  > Rate limit reached. Sleeping for {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
            start_time = time.time()
            requests_this_minute = 0
        
        prompt_text = row.Prompt
        final_response = generate_structured_response(prompt_text)

        requests_this_minute += 1
        
        print(f"  > Generated Response: {repr(final_response[:100])}...")
        results.append({'Master_Index': row.Master_Index, 'Clinician': final_response})
        print("-" * 20)
            
    print("Prediction process finished.")
    
    submission_df = pd.DataFrame(results)
    submission_df.to_csv('submission.csv', index=False)
    
    print("Submission file 'submission.csv' has been created successfully!")
    print(submission_df.head())
else:
    print("Execution halted because Gemini model could not be initialized.")


Starting prediction process with 'Moonshot' architecture...
Processing row 1/100: Master_Index = ID_CUAOY


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  > Generated Response: 'summary: A 24-year-old female presents with a 2-day history of gradually worsening sharp pain on the'...
--------------------
Processing row 2/100: Master_Index = ID_OGSAY


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  > Generated Response: 'Error: An unexpected API error occurred: 500 An internal error has occurred. Please retry or report '...
--------------------
Processing row 3/100: Master_Index = ID_TYHSA


...

  > Generated Response: 'summary: A 35-year-old man presents with primary infertility, having been trying to conceive with hi'...
--------------------
Processing row 99/100: Master_Index = ID_SNZBL


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  > Generated Response: 'summary: A 48-year-old patient post-thyroidectomy presents with acute onset dizziness, loss of voice'...
--------------------
Processing row 100/100: Master_Index = ID_KXLGG


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  > Generated Response: 'summary: A 58-year-old patient presented with fever, chills, and generalized lymphadenopathy, leadin'...
--------------------
Prediction process finished.
Submission file 'submission.csv' has been created successfully!
  Master_Index                                          Clinician
0     ID_CUAOY  summary: A 24-year-old female presents with a ...
1     ID_OGSAY  Error: An unexpected API error occurred: 500 A...
2     ID_TYHSA  summary: A 22-year-old man presents to a natio...
3     ID_CZXLD  summary: A 6-year-old girl presented with a hi...
4     ID_ZJQUQ  summary: A 1-year-old unvaccinated boy present...
